### A system of advection - diffusion - reaction equations

The problems we have encountered so far—with the notable exception of the Navier -
Stokes equations—all share a common feature: they all involve models expressed by a single
scalar or vector PDE. In many situations the model is instead expressed as a system of
PDEs, describing different quantities possibly governed by (very) different physics. As we
saw for the Navier - Stokes equations, one way to solve a system of PDEs in FEniCS is to
use a splitting method where we solve one equation at a tme and feed the solution from
one equation into the next. However, one of the strengths with FEniCS is the ease by
which one can instead define variational problems that couple several PDEs into one
compound system. In this section, we will look at how to use FEniCS to write solvers for
such systems of coupled PDEs. The goal is to demonstrate how easy it is to implement
fully implicit, also known as monolithic, solvers in FEniCS.

#### PDE problem

Our model problem is the following system of advection - diffusion - reaction equations:

\begin{align}
\dfrac{\partial u_1}{\partial t} + w\cdot \nabla u_1 - \nabla\cdot   (\epsilon\nabla u_1) &= f_1 - K u_1 u_2,
\tag{1}
\label{eq:adr_equation_1}
\end{align}

\begin{align}
\dfrac{\partial u_2}{\partial t} + w\cdot\nabla u_2 - \nabla\cdot (\epsilon\nabla u_2) &= f_2 - K u_1 u_2,
\tag{2}
\label{eq:adr_equation_2}
\end{align}

\begin{align}
\dfrac{\partial u_3}{\partial t} + w\cdot\nabla u_3 - \nabla\cdot (\epsilon\nabla u_3) &=f_3 + K u_1 u_2 - K u_3.
\tag{3}
\label{eq:adr_equation_3}
\end{align}

This system models the chemical reaction between two species $A$ and $B$ in some domain $\Omega$:

\begin{align}
A+B\rightarrow C.
\end{align}

We assume that the reaction is *first-order*, meaning that the reaction rate is proportional to the concentrations $[A]$ and $[B]$ of the two species $A$ and $B$:

\begin{align}
\dfrac{d}{dt}[C]&=K[A][B]
\end{align}

We also assume that the formed species $C$ spontaneously decays with a rate proportional to the concentration $[C]$. In the PDE systems \eqref{eq:adr_equation_1} - \eqref{eq:adr_equation_3}, we use the variables $u_1$, $u_2$, $u_3$ to denote the concentrations of the three species:

\begin{align}
u_1=[A],\quad u_2=[B],\quad u_3=[C].
\end{align}

We see that the chemical reactions are accounted for in the right-hand sides of the PDE system \eqref{eq:adr_equation_1} - \eqref{eq:adr_equation_3}.

The chemical reactions take part at each point in the domain $\Omega$. In addition, we assume that the species $A$, $B$, $C$ diffuse throughout the domain with the diffusivity $\epsilon$ (the terms $-\nabla\cdot (\epsilon\nabla u_i)$ and are advected with velocity $w$ (the terms $w\cdot\nabla u_i$).

We assume that $u_1=u_2=u_3=0$ at $t=0$ and inject the species $A$ and $B$ into the system by specifying nonzero source terms $f_1$ and $f_2$ close to the corners at the inflow and take $f_3=0$. The result will be that $A$ and $B$ are convected by advectionand diffusion throughout the channel, and when they mix the species $C$ will be formed.

Since the system is one-way coupled from the Navier - Stokes subsystem to the advection - diffusion - reaction subsystem, we do not need to recompute the soluton to the Navier - Stokes equations, but can just read back the previously computed velocity field $w$ and feed it into our equations. But we *do* need to learn how to read and write solutions for time dependent PDE problems.

#### Variational Formulation

We obtain the variational formulation of our system by multiplying each equation by a test function, integrating the second-order term $-\nabla\cdot (\epsilon\nabla u_i)$ by parts, and summing up the equations. When working with FEniCS it is convenient to think of the PDE system as a vector of equations. The test functions are collected in a vector too, and the variational formulation is the inner product of the vector PDE and the vector test function.

We also need to introduce some discretization in time. We will use the backward Euler method as before when we solved the heat equation and approximate the time derivatives $(u_i^{n+1} - u_i^n)/\Delta t$. Let $v_1$, $v_2$, and $v_3$ be the test functions, or the components of the test vector function. The inner product results in
\begin{align}
\int_{\Omega}(\Delta t^{-1}(u_1^{n+1} - u_1^n) v_1 + w\cdot \nabla u_1^{n+1}v_1 + \epsilon\nabla u_1^{n+1}\cdot \nabla v_1)\ dx\\
+\int_{\Omega}(\Delta t^{-1}(u_2^{n+1} - u_2^n) v_2 + w\cdot \nabla u_2^{n+1}v_2 + \epsilon\nabla u_2^{n+1}\cdot \nabla v_2)\ dx\\
+\int_{\Omega}(\Delta t^{-1}(u_3^{n+1} - u_3^n) v_3 + w\cdot \nabla u_3^{n+1}v_3 + \epsilon\nabla u_3^{n+1}\cdot \nabla v_3)\ dx\\
-\int_{\Omega}(f_1 v_1 + f_2 v_2 + f_3 v_3)\ dx\\
-\int_{\Omega}(-Ku_1^{n+1}u_2^{n+1}v_1 -Ku_1^{n+1}u_2^{n+1}v_2 + +Ku_1^{n+1}u_2^{n+1}v_3 - Ku_3^{n+1}v_3)\ dx &= 0.
\tag{4}
\label{eq:adr_equation_variational}
\end{align}

For this problem it is natural to assume homogeneous Neumann boundary conditions for $u_1$, $u_2$, and $u_3$; that is $\partial u_i/\partial n = 0$  for $i=1,2,3$. This means that the boundary terms vanish when we integrate by parts.

#### FEniCS Implementation

The first step is to read the mesh from file. Luckily, we made sure to save the mesh to file in the Navier - Stokes example and can now easily read it back from file:

```python
mesh = Mesh('navier_stokes_cylinder/cylinder.xml.gz')
```

The mesh is stores in the native FEniCS XML format (with additional gzipping to decrease the file size).

Next, we need to define the finite element function space. For this problem, we need to define several spaces. The first space we create is the space for the velocity field $w$ from the Navier - Stokes simulation. we call this space $W$ and define the space by

```python
W = VectorFunctionSpace(mesh, 'P', 2)
```

It is important that this space is exactly the same space that we used for the velocity field in the Navier - Stokes solver. To read the values for the velocity field, we use a ```TimeSeries```.

```python
timeseries_w = TimeSeries('navier_stokes_cylinder/cylinder.xml.gz')
```

This will initialize the object ```timeseries_w``` which we will call later in the time-stepping loop to retrieve values from the file ```velocity_series.h5``` (in binary HDF5 format).

For the three concentrations $u_1$, $u_2$, and $u_3$, we want to create a *mixed space* with functions that represent the full system $(u_1,u_2,u_3)$ as a single entity. To do this, we need to define a ```MixedElement``` as the product space of three simple finite elements and then use the mixed element to define the function space:

```python
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1])
V=FunctionSpace(mesh, element)
```

<blockquote>
<markdown>    
    
**Mixed elements as products of elements**
    
FEniCS also allows finite elements to be defined as products of simple elements (or mixed elements). For example, the well-known Taylor - Hood element, with quadratic velocity components and linear pressure functions, may be defined as follows:
```python
P2 = VectorElement('P', triangle, 2)
P1 = FiniteElement('P', triangle, 1)
TH = P2 * P1
```
The syntax works great for two elements, but for three element we meet a subtle issue how the Python interpreter handles the ```*``` operator. For the reaction system, we create the mixed element by ```element = MixedElement([P1, P1, P1])``` and one would be tempted to write
```python
element = P1 * P1 * P1    
```
However, this is equivalent to writing ```element = (P1 * P1) * P1 so the result will be a mixed element consisting of two subsystems, the first of which in turn consists of two scalar subsystems.
    
Finally, we remark that for the simple case of a mixed system consisting of three scalar elements for the reaction system, the definition is in fact equivalent to using a standard vector-valued element:
```python
element = VectorElement('P', triangle, 1, dim=3)
V = FunctionSpace(mesh, element)
```
</markdown>
</blockquote>

Once the space has been created, we need to define our test functions and finite element functions. Test functions for a mixed function space can be created by replacing ```TestFunction``` by ```TestFunctions```:

```python
v_1, v_2, v_3 = TestFunctions(V)
```

Since the problem is nonlinear, we need to work with functions rather than trial functions for the unknowns. This can be done by using the corresponding ```Functions``` construction in FEniCS. However, as we will need to access the ```Function``` for the entire system itself,
we first need to create that function and then access its components:

```python
u = Function(V)
u_1, u_2, u_3 = split(u)
```

These functions will be used to represent $u_1$, $u_2$, and $u_3$ at the new time level $n+1$. The corresponding values at the previous time level $n$ are denoted by ```u_n1```, ```u_n2```, and ```u_n3``` in our program.

When now all functions and test functions have been defines, we can express the nonlinear variational problem \eqref{eq:adr_equation_variational}:

```python
F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
  + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
  + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
  + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
  + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
  + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
  - f_1*v_1*dx - f_2*v_2*dx - f_3*v_3*dx
```

The time-stepping simply consists of solving this variational problem in each time step by a call to the ```solve``` function:

```python
t = 0
for n in range(num_steps):
    t += dt
    timeseries_w.retrieve(w.vector(), t)
    solve(F == 0, u)
    u_n.assign(u)
```

In each time step, we first read the current value for the velocity field from the time series we have previously stored. We then solve the nonlinear system, and assign the computed values to the left-hand side values for the next time interval. When retrieving values from a ```TimeSeries```, the values will by default be interpolated (linearly) to the given time ```t``` if the time does not exactly match a sample in the series.

The complete code is presented below.

In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation

T = 5.0            # final time
num_steps = 500    # number of time steps
dt = T / num_steps # time step size
eps = 0.01         # diffusion coefficient
K = 10.0           # reaction rate

# Read mesh from file
mesh = Mesh('navier_stokes_cylinder/cylinder.xml.gz')

# Define function space for velocity
W = VectorFunctionSpace(mesh, 'P', 2)

# Define function space for system of concentrations
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1])
V = FunctionSpace(mesh, element)

# Define test functions
v_1, v_2, v_3 = TestFunctions(V)

# Define functions for velocity and concentrations
w = Function(W)
u = Function(V)
u_n = Function(V)

# Split system functions to access components
u_1, u_2, u_3 = split(u)
u_n1, u_n2, u_n3 = split(u_n)

# Define source terms
f_1 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.1,2)<0.05*0.05 ? 0.1 : 0',
degree=1)
f_2 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.1 : 0',
degree=1)
f_3 = Constant(0)

# Define expressions used in variational forms
k = Constant(dt)
K = Constant(K)
eps = Constant(eps)

# Define variational problem
F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
  + eps*dot(grad(u_1), grad(v_1))*dx + K*u_1*u_2*v_1*dx \
  + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
  + eps*dot(grad(u_2), grad(v_2))*dx + K*u_1*u_2*v_2*dx \
  + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
  + eps*dot(grad(u_3), grad(v_3))*dx - K*u_1*u_2*v_3*dx + K*u_3*v_3*dx \
  - f_1*v_1*dx - f_2*v_2*dx - f_3*v_3*dx

# Create time series for reading velocity data
timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series')

# Create VTK files for visualization output
vtkfile_u_1 = File('reaction_system/u_1.pvd')
vtkfile_u_2 = File('reaction_system/u_2.pvd')
vtkfile_u_3 = File('reaction_system/u_3.pvd')

# Time-stepping
t = 0
for n in range(num_steps):
    
    # Update current time
    t += dt
    
    # Read velocity from file
    timeseries_w.retrieve(w.vector(), t)
    
    # Solve variational problem for time step
    solve(F == 0, u)
    
    # Save solution to file (VTK)
    _u_1, _u_2, _u_3 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    vtkfile_u_3 << (_u_3, t)
    
    # Update previous solution
    u_n.assign(u)